In [18]:
import tensorflow as tf
import numpy as np
import config
from model_utils import Chatbot
from cornell_data_utils import *


In [19]:
from tqdm import tqdm

In [3]:
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

In [4]:
cleaned_questions, cleaned_answers = clean_data()

In [5]:
vocab, word_to_id, id_to_word = create_vocab(cleaned_questions, cleaned_answers)

In [6]:
encoded_questions = encoder(cleaned_questions, word_to_id)

In [7]:
encoded_answers = encoder(cleaned_answers, word_to_id, True)

In [8]:
bucketed_data = bucket_data(encoded_questions, encoded_answers, word_to_id)

In [20]:
model = Chatbot(config.LEARNING_RATE, 
                config.BATCH_SIZE, 
                config.ENCODING_EMBED_SIZE, 
                config.DECODING_EMBED_SIZE, 
                config.RNN_SIZE, 
                config.NUM_LAYERS,
                len(vocab), 
                word_to_id, 
                config.CLIP_RATE) #4=clip_rate

Instructions for updating:
Use the retry module or similar alternatives.


In [21]:
session = tf.Session()

In [22]:
session.run(tf.global_variables_initializer())

In [23]:
saver = tf.train.Saver(max_to_keep=10)

In [24]:
for i in range(config.EPOCHS):
    epoch_accuracy = []
    epoch_loss = []
    for b in range(len(bucketed_data)):
        bucket = bucketed_data[b]
        questions_bucket = []
        answers_bucket = []
        bucket_accuracy = []
        bucket_loss = []
        for k in range(len(bucket)):
            questions_bucket.append(np.array(bucket[k][0]))
            answers_bucket.append(np.array(bucket[k][1]))
            
        for ii in tqdm(range(len(questions_bucket) //  config.BATCH_SIZE)):
            
            starting_id = ii * config.BATCH_SIZE
            
            X_batch = questions_bucket[starting_id:starting_id+config.BATCH_SIZE]
            y_batch = answers_bucket[starting_id:starting_id+config.BATCH_SIZE]
            
            feed_dict = {model.inputs:X_batch, 
                         model.targets:y_batch, 
                         model.keep_probs:config.KEEP_PROBS, 
                         model.decoder_seq_len:[len(y_batch[0])]*config.BATCH_SIZE,
                         model.encoder_seq_len:[len(X_batch[0])]*config.BATCH_SIZE}
            
            cost, _, preds = session.run([model.loss, model.opt, model.predictions], feed_dict=feed_dict)
            
            epoch_accuracy.append(get_accuracy(np.array(y_batch), np.array(preds)))
            bucket_accuracy.append(get_accuracy(np.array(y_batch), np.array(preds)))
            
            bucket_loss.append(cost)
            epoch_loss.append(cost)
            
        print("Bucket {}:".format(b+1), 
              " | Loss: {}".format(np.mean(bucket_loss)), 
              " | Accuracy: {}".format(np.mean(bucket_accuracy)))
        
    print("EPOCH: {}/{}".format(i, config.EPOCHS), 
          " | Epoch loss: {}".format(np.mean(epoch_loss)), 
          " | Epoch accuracy: {}".format(np.mean(epoch_accuracy)))
    
    saver.save(session, "checkpoint/chatbot_{}.ckpt".format(i))

100%|██████████████████████████████████████████████████████████████████████████████| 1430/1430 [11:33<00:00,  2.06it/s]


Bucket 1:  | Loss: 2.668109178543091  | Accuracy: 0.4921081002331002


100%|████████████████████████████████████████████████████████████████████████████████| 779/779 [10:15<00:00,  1.27it/s]


Bucket 2:  | Loss: 2.890730857849121  | Accuracy: 0.4076997753530167


100%|████████████████████████████████████████████████████████████████████████████████| 739/739 [18:46<00:00,  1.52s/it]


...

Bucket 3:  | Loss: 1.651945948600769  | Accuracy: 0.5584968425800632


100%|████████████████████████████████████████████████████████████████████████████████| 353/353 [13:16<00:00,  2.26s/it]


Bucket 4:  | Loss: 1.2978712320327759  | Accuracy: 0.6552489081680831


100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [06:30<00:00,  3.79s/it]


Bucket 5:  | Loss: 1.6371400356292725  | Accuracy: 0.5851061893203884
EPOCH: 49/50  | Epoch loss: 1.6344558000564575  | Epoch accuracy: 0.5035619399970622
